In [22]:
import logging
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import requests
import sys
import time
from selenium.webdriver.common.keys import Keys
import urllib.parse as urlparse
from urllib.parse import parse_qs
import pandas as pd
sys.path.append('../src/')
from util.selenium_dispatcher import SeleniumDispatcher
from config import Config

In [8]:
 def get_request_token(kite_url):
        print('Starting to fetch request token for Kite API')
        selenium = SeleniumDispatcher(headless=True)
        driver = selenium.get_driver()
        driver.get(kite_url)
        time.sleep(4)
        username_field = driver.find_element_by_xpath("//input[@type='text']")
        username_field.send_keys(Config.KITE_USER_ID)
        password_field = driver.find_element_by_xpath("//input[@type='password']")
        password_field.send_keys(Config.KITE_PASSWORD)
        password_field.send_keys(Keys.ENTER)
        pin_field = driver.find_element_by_xpath("//input[@type='password']")
        pin_field.send_keys(Config.KITE_PIN)
        pin_field.send_keys(Keys.ENTER)
        url = driver.current_url
        parsed = urlparse.urlparse(url)
        token = parse_qs(parsed.query)['request_token'][0]
        print('Request token received!')
        selenium.destroy_driver()
        return token

In [3]:
kite = KiteConnect(api_key=Config.KITE_API_KEY)

In [4]:
print(kite.login_url())

https://kite.trade/connect/login?api_key=37wk1ehie3noonc1&v=3


In [5]:
data = kite.generate_session(get_request_token(kite.login_url()), api_secret=Config.KITE_API_SECRET)

Starting to fetch request token for Kite API
Request token received!


In [6]:
kite.set_access_token(data["access_token"])


In [7]:
instruments = kite.instruments(exchange='NFO')

In [8]:
instruments[0]

OrderedDict([('instrument_token', 11260674),
             ('exchange_token', '43987'),
             ('tradingsymbol', 'ACC20MAR1080CE'),
             ('name', 'ACC'),
             ('last_price', 0.0),
             ('expiry', datetime.date(2020, 3, 26)),
             ('strike', 1080.0),
             ('tick_size', 0.05),
             ('lot_size', 400),
             ('instrument_type', 'CE'),
             ('segment', 'NFO-OPT'),
             ('exchange', 'NFO')])

In [17]:
required_instruments = list()

In [18]:
for instrument in instruments:
    if 'BANKNIFTY20312' in instrument['tradingsymbol']:
        required_instruments.append(instrument)

In [20]:
len(required_instruments)

210

In [23]:
df = pd.DataFrame.from_dict(required_instruments)

In [31]:
df_put = df[df.instrument_type == 'PE'].reset_index().drop('index', axis=1)

In [32]:
df_call = df[df.instrument_type == 'CE'].reset_index().drop('index', axis=1)

In [11]:
# Initialise
kws = KiteTicker(Config.KITE_API_KEY, data['access_token'])
tick_count = 0

In [12]:
def on_ticks(ws, ticks):
    # Callback to receive ticks.
    global tick_count
    print("Ticks: {}".format(ticks))
    tick_count = tick_count + 1
    print("Length: {}".format(tick_count))
#     if tick_count > 5:
#         ws.close()

def on_connect(ws, response):
    pass
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe([11260674])
#     ws.subscribe([134657])
    # Set RELIANCE to tick in `full` mode.
#     ws.set_mode(ws.MODE_LTP, [134657, 408065, 237569, 389377, 2791681])
    ws.set_mode(ws.MODE_FULL, [11260674])

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

In [13]:
# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

In [14]:
# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect()

Ticks: [{'tradable': True, 'mode': 'full', 'instrument_token': 11260674, 'last_price': 261.7, 'last_quantity': 400, 'average_price': 232.52, 'volume': 1600, 'buy_quantity': 10000, 'sell_quantity': 10000, 'ohlc': {'open': 203.35, 'high': 261.7, 'low': 203.35, 'close': 387.3}, 'change': -32.4296411050865, 'last_trade_time': datetime.datetime(2020, 3, 6, 15, 13, 32), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2020, 3, 6, 17, 20, 38), 'depth': {'buy': [{'quantity': 800, 'price': 203.45, 'orders': 1}, {'quantity': 9200, 'price': 203.3, 'orders': 1}, {'quantity': 0, 'price': 0.0, 'orders': 0}, {'quantity': 0, 'price': 0.0, 'orders': 0}, {'quantity': 0, 'price': 0.0, 'orders': 0}], 'sell': [{'quantity': 800, 'price': 257.85, 'orders': 1}, {'quantity': 9200, 'price': 261.75, 'orders': 1}, {'quantity': 0, 'price': 0.0, 'orders': 0}, {'quantity': 0, 'price': 0.0, 'orders': 0}, {'quantity': 0, 'price': 0.0, 'orders': 0}]}}]
Length: 1


Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Unhandled Error
Traceback (most recent call last):
  File "/Users/mayank.gupta/venv/nse/lib/python3.7/site-packages/twisted/internet/defer.py", line 311, in addCallbacks
    self._runCallbacks()
  File "/Users/mayank.gupta/venv/nse/lib/python3.7/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/Users/mayank.gupta/venv/nse/lib/python3.7/site-packages/twisted/internet/base.py", line 447, in _continueFiring
    callable(*args, **kwargs)
  File "/Users/mayank.gupta/venv/nse/lib/python3.7/site-packages/twisted/internet/base.py", line 706, in disconnectAll
    failure.Failure(main.CONNECTION_LOST))
--- <exception caught here> ---
  File "